# Import necessary packages

In [1]:
import copy
import sympy as sp
from sympy.matrices import ones, eye
import re
import dimod
import neal
from dwave.system import DWaveSampler, EmbeddingComposite
from rdkit import Chem
import py3Dmol

In [2]:
mol_file = './Molecules/25A12Rot_2.mol2'
mol = Chem.MolFromMol2File(mol_file)
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Input Variables

coords_original - Coordinates considered for distance measure in the hubo<br>torsional_bonds - Torsional bond numbers and their respective coordinate ends<br>coords_rotation_dict - Coordinate numbers and the bonds that affect them<br>distance_pairs - Pairs of coordinates to find distance

In [7]:
print('No. of atoms: ', mol.GetNumAtoms())
conformers = mol.GetConformers()
coords_original = {}
for i in range(n_atoms):
    coords_original[i] = list(conformers[0].GetAtomPosition(i))
print('Coordinates of atoms:\n', coords_original)

No. of atoms:  25
Coordinates of atoms:
 {0: [12.305, -0.778, -0.508], 1: [11.0, -0.01, -0.373], 2: [9.839, -0.938, -0.008], 3: [8.526, -0.154, 0.13], 4: [7.333, -1.002, 0.304], 5: [7.385, -1.885, 1.482], 6: [7.42, -1.187, 2.849], 7: [7.463, -2.211, 3.987], 8: [7.494, -1.531, 5.347], 9: [6.114, -0.178, 0.22], 10: [4.808, -0.978, 0.227], 11: [3.609, -0.075, -0.057], 12: [2.435, -0.883, 0.009], 13: [1.208, -0.308, -0.214], 14: [1.06, 1.058, -0.506], 15: [-0.199, 1.601, -0.706], 16: [-1.31, 0.782, -0.608], 17: [-1.17, -0.587, -0.317], 18: [-2.677, -1.439, -0.213], 19: [0.095, -1.206, -0.116], 20: [0.35, -2.78, 0.217], 21: [-0.849, -3.648, 0.219], 22: [1.181, -3.165, 1.47], 23: [2.134, -3.924, 0.685], 24: [1.449, -3.574, -0.544]}


In [8]:
coords_median_original = {}
coords_median_original[0] = coords_original[2]
coords_median_original[1] = coords_original[6]
coords_median_original[2] = coords_original[9]
coords_median_original[3] = coords_original[10]
coords_median_original[4] = coords_original[14]
coords_median_original[5] = coords_original[19]
coords_median_original[6] = coords_original[20]
coords_median_original[7] = coords_original[23]

coords_dict = copy.deepcopy(coords_median_original)

n_atoms = len(coords_dict)

torsional_bonds = {
    0: (2, 3),
    1: (5, 6)
}

coords_rotation_dict = {}
for i in range(n_atoms):
    coords_rotation_dict[i] = []
coords_rotation_dict[4] = [0]
coords_rotation_dict[5] = [0]
coords_rotation_dict[6] = [0]
coords_rotation_dict[7] = [0, 1]

distance_pairs_indices = {
    0: [4, 5, 6, 7],
    1: [4, 5, 6, 7],
    2: [4, 5, 6, 7],
    3: [6, 7],
    4: [6, 7],
    5: [7]
}

distance_pairs = []
for x, y in distance_pairs_indices.items():
    for i in y:
        distance_pairs.append((x, i))
print(distance_pairs)


[(0, 4), (0, 5), (0, 6), (0, 7), (1, 4), (1, 5), (1, 6), (1, 7), (2, 4), (2, 5), (2, 6), (2, 7), (3, 6), (3, 7), (4, 6), (4, 7), (5, 7)]


In [9]:
n_bonds = len(torsional_bonds)  # no. of bonds
n_angles = 8  # no. of discrete angles

x = sp.symbols(f'x(0:{n_bonds*n_angles})')  # hubo variables
print(x)

(x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15)


# Function to generate hard constraint

In [10]:
def generate_hard_constraint():
    hard_constraint = 0
    index = 0
    for i in range(n_bonds):
        summation = 0
        for j in range(n_angles):
            summation += x[index]
            index += 1
        hard_constraint += (summation - 1) ** 2
    a_const = sp.Symbol('A_const')
    hard_constraint *= a_const
    return hard_constraint

In [11]:
hard_hubo = generate_hard_constraint()
sp.pprint(hard_hubo)

        ⎛                                           2                         
A_const⋅⎝(x₀ + x₁ + x₂ + x₃ + x₄ + x₅ + x₆ + x₇ - 1)  + (x₁₀ + x₁₁ + x₁₂ + x₁₃

                           2⎞
 + x₁₄ + x₁₅ + x₈ + x₉ - 1) ⎠


# Functions to rotate coordinates with hubo variables

In [12]:
def rotate_coordinates(rotation_matrix, old_coords):
    coord_vector = ones(4, 1)
    coord_vector[0, 0] = old_coords[0]
    coord_vector[1, 0] = old_coords[1]
    coord_vector[2, 0] = old_coords[2]
    coord_rot_vector = sp.expand(rotation_matrix * coord_vector)
    return [coord_rot_vector[0, 0], coord_rot_vector[1, 0], coord_rot_vector[2, 0]]

In [13]:
def generate_thetas():
    angle_incr = 2 * sp.pi / n_angles
    thetas = [i*angle_incr for i in range(n_angles)]
    return thetas

In [14]:
def generate_rotation_matrix(first_coords, second_coords, bond_no):
    x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
    x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
    dx = x_ddash - x_dash
    dy = y_ddash - y_dash
    dz = z_ddash - z_dash
    l_sq = dx ** 2 + dy ** 2 + dz ** 2
    l = sp.sqrt(l_sq)
    thetas = generate_thetas()
    c_theta = 0.0
    s_theta = 0.0
    index = n_angles * bond_no
    for i in range(n_angles):
        c_theta += sp.cos(thetas[i]) * x[index]
        s_theta += sp.sin(thetas[i]) * x[index]
        index += 1
    rotation_matrix = eye(4)
    rotation_matrix[0, 0] = (dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[0, 1] = (dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq
    rotation_matrix[0, 2] = (dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq
    rotation_matrix[0, 3] = ((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (
            y_dash * dz - z_dash * dy) * l * s_theta) / l_sq
    rotation_matrix[1, 0] = (dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq
    rotation_matrix[1, 1] = (dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq
    rotation_matrix[1, 2] = (dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq
    rotation_matrix[1, 3] = ((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (
            z_dash * dx - x_dash * dz) * l * s_theta) / l_sq
    rotation_matrix[2, 0] = (dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq
    rotation_matrix[2, 1] = (dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq
    rotation_matrix[2, 2] = (dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq
    rotation_matrix[2, 3] = ((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (
            x_dash * dy - y_dash * dx) * l * s_theta) / l_sq
    return rotation_matrix

In [15]:
def rotate_all_coordinates(coords):
    for i in coords.keys():
        if len(coords_rotation_dict[i]) > 0:
            rot_mat = eye(4, 4)
            for bond_no in coords_rotation_dict[i]:
                temp_rot_mat = generate_rotation_matrix(coords[torsional_bonds[bond_no][0]],
                                                        coords[torsional_bonds[bond_no][1]], bond_no)
                rot_mat = rot_mat * temp_rot_mat
            coords_dict[i] = rotate_coordinates(rot_mat, coords[i])

In [16]:
rotate_all_coordinates(coords_median_original)

In [17]:
sp.pprint(coords_median_original)

{0: [9.839, -0.938, -0.008], 1: [7.42, -1.187, 2.849], 2: [6.114, -0.178, 0.22
], 3: [4.808, -0.978, 0.227], 4: [1.06, 1.058, -0.506], 5: [0.095, -1.206, -0.
116], 6: [0.35, -2.78, 0.217], 7: [2.134, -3.924, 0.685]}


In [18]:
sp.pprint(coords_dict)

{0: [9.839, -0.938, -0.008], 1: [7.42, -1.187, 2.849], 2: [6.114, -0.178, 0.22
], 3: [4.808, -0.978, 0.227], 4: [-1.93240675453013⋅x₀ - 0.77941771216583⋅√2⋅x
₁ + 0.373571330198466⋅x₂ + 1.1529890423643⋅√2⋅x₃ + 1.93240675453013⋅x₄ + 0.779
41771216583⋅√2⋅x₅ - 0.373571330198466⋅x₆ - 1.1529890423643⋅√2⋅x₇ + 2.992406754
53013, 3.14815512739349⋅x₀ + 1.25298922232754⋅√2⋅x₁ - 0.642176682738413⋅x₂ - 1
.89516590506595⋅√2⋅x₃ - 3.14815512739349⋅x₄ - 1.25298922232754⋅√2⋅x₅ + 0.64217
6682738413⋅x₆ + 1.89516590506595⋅√2⋅x₇ - 2.09015512739349, -0.742731357364693⋅
x₀ - 2.2183077466491⋅√2⋅x₁ - 3.69388413593351⋅x₂ - 1.47557638928441⋅√2⋅x₃ + 0.
742731357364693⋅x₄ + 2.2183077466491⋅√2⋅x₅ + 3.69388413593351⋅x₆ + 1.475576389
28441⋅√2⋅x₇ + 0.236731357364693], 5: [-1.18578250830781⋅x₀ - 0.502788524746953
⋅√2⋅x₁ + 0.180205458813901⋅x₂ + 0.682993983560854⋅√2⋅x₃ + 1.18578250830781⋅x₄ 
+ 0.502788524746953⋅√2⋅x₅ - 0.180205458813901⋅x₆ - 0.682993983560854⋅√2⋅x₇ + 1
.28078250830781, 1.93262327209323⋅x₀ + 0.80929915178

# Functions to generate the optimization contraint in hubo

In [20]:
def distance_squared(first_coords, second_coords):
    dis_sq = sp.expand((second_coords[0] - first_coords[0])**2 + (second_coords[1] - first_coords[1])**2 +(second_coords[2] - first_coords[2])**2)
    return dis_sq

def generate_distance_hubo():
    distance_sq = 0
    for pair in distance_pairs:
        distance_sq += distance_squared(coords_dict[pair[0]], coords_dict[pair[1]])
    return distance_sq.expand()

In [21]:
import time
start = time.time()

distance_hubo = generate_distance_hubo()

end = time.time()
print("The time of execution: ", (end-start), "s")

The time of execution:  84.96678805351257 s


# The full hubo expression is written in the file 'full_hubo_expr.txt'

In [22]:
hubo_expr = sp.expand(hard_hubo - distance_hubo)
hubo_expr

**2*x4*x6 + 8.96586147318318*sqrt(2)*x13**2*x4*x7 - 1.97758476261356e-16*x13**2*x4 - 8.96586147318318*x13**2*x5**2 - 8.96586147318318*sqrt(2)*x13**2*x5*x6 - 1.11022302462516e-15*x13**2*x5*x7 - 1.24032728532342e-16*sqrt(2)*x13**2*x5 - 8.96586147318318*x13**2*x6**2 - 8.96586147318318*sqrt(2)*x13**2*x6*x7 - 1.14014700458576e-15*x13**2*x6 - 8.96586147318317*x13**2*x7**2 + 2.57606436182556e-16*sqrt(2)*x13**2*x7 - 6.1475390307156*x13**2 - 15.4657544155233*sqrt(2)*x13*x14*x2**2 - 30.9315088310466*x13*x14*x2*x3 - 2.35922392732846e-15*sqrt(2)*x13*x14*x2*x4 + 30.9315088310466*x13*x14*x2*x5 + 30.9315088310466*sqrt(2)*x13*x14*x2*x6 + 30.9315088310466*x13*x14*x2*x7 + 1.8101839471818e-15*sqrt(2)*x13*x14*x2 - 15.4657544155233*sqrt(2)*x13*x14*x3**2 - 30.9315088310466*x13*x14*x3*x4 + 2.22044604925031e-15*sqrt(2)*x13*x14*x3*x5 + 30.9315088310466*x13*x14*x3*x6 + 30.9315088310466*sqrt(2)*x13*x14*x3*x7 - 9.92261828258734e-16*x13*x14*x3 - 15.4657544155233*sqrt(2)*x13*x14*x4**2 - 30.9315088310466*x13*x14*x4*

In [23]:
def hubo_expr_to_dict():
    hubo_args = hubo_expr.args
    hubo_dict = {}
    for monom in hubo_args:
        dict_value = monom.as_coeff_mul()[0]
        monom_key = []
        monom_coeffs = monom.as_coeff_mul()[1]
        for monom_item in monom_coeffs:
            if re.match("^x(\d*)\d$", str(monom_item)):
                monom_key.append(int(str(monom_item)[1:]))
            elif re.match("^x(\d*)\\*\\*(\d)$", str(monom_item)):
                monom_key.append(int(str(monom_item).split('**')[0][1:]))
            else:
                dict_value *= monom_item
        if len(monom_key) > 0:
            monom_key.sort()
            dict_key = tuple(monom_key)
        else:
            dict_key = ()
        dict_value = dict_value.evalf()
        if dict_key in hubo_dict:
            hubo_dict[dict_key] += dict_value
        else:
            hubo_dict[dict_key] = dict_value
    return hubo_dict

In [24]:
hubo_dict = hubo_expr_to_dict()
print(hubo_dict)

6223787898086, (4, 8, 15): -2.46223787898086, (2, 11, 12): -2.46223787898086, (2, 8, 15): -2.46223787898086, (4, 12, 13): -3.48740614070350, (4, 8, 9): -3.48740614070350, (6, 12, 13): -3.48740614070350, (6, 8, 9): -3.48740614070350, (0, 10, 11): -20.8467113850420, (0, 14, 15): -20.8467113850420, (2, 10, 11): -20.8467113850420, (2, 14, 15): -20.8467113850420, (6, 9, 10): -21.8718796467647, (6, 13, 14): -21.8718796467647, (1, 2, 11, 12): -3.48213020224350, (1, 6, 8, 11): -3.48213020224350, (1, 6, 12, 15): -3.48213020224350, (1, 2, 8, 15): -3.48213020224350, (5, 6, 11, 12): -3.48213020224350, (2, 5, 8, 11): -3.48213020224350, (2, 5, 12, 15): -3.48213020224350, (5, 6, 8, 15): -3.48213020224350, (1, 2, 10, 11): -29.4817019716040, (1, 6, 10, 15): -29.4817019716040, (1, 6, 11, 14): -29.4817019716040, (1, 2, 14, 15): -29.4817019716040, (2, 3, 10, 11): -29.4817019716040, (5, 6, 10, 11): -29.4817019716040, (6, 7, 10, 11): -29.4817019716040, (2, 5, 10, 15): -29.4817019716040, (2, 7, 10, 15): -29.

In [25]:
file_name = "hubo_dict.txt"
f = open(file_name, "w")
print('full hubo dictionary written in file - hubo_dict.txt')
f.write(str(hubo_dict))
f.close()

full hubo dictionary written in file - hubo_dict.txt


In [26]:
#This is used to check the maximum coefficient appearing in Hubo_B
A_const=0
read_dictionary_B = open(file_name, 'r').read()
HUBO_B=eval(read_dictionary_B)

#We set the Hard constraint strength as the (maximum coefficient appearing in Hubo_B)*const.
#const was empirically selected to be 10
const=10
A_const=max(map(abs, list(HUBO_B.values())))*const
# A_const = 1000

#read the final HUBO
read_dictionary= open(file_name, 'r').read()
HUBO=eval(read_dictionary)

In [27]:
print("Current size of the HUBO:",len(HUBO)) 

Current size of the HUBO: 1369


In [28]:
def threshold_approx(h, val=1):
    d =h.copy()
    monoms = h.keys()
    for m in monoms:     
        temp = d[m]
        if (temp < 0.0):
            temp = -1.0 * temp
        if (temp <= (10.0 ** (val))):
            del d[m]
    return d

In [29]:
#Coefficints with absolute value less than 10^{threshold} are deleted from the HUBO.
threshold=2

HUBO=threshold_approx(HUBO,threshold)

print("Size of the HUBO after threshold approximation:",len(HUBO))

Size of the HUBO after threshold approximation: 73


In [30]:
#calculate the strength parameter needed by make_quadratic
max_hubo_value=max(map(abs, list(HUBO.values())))
strength=1.5*max_hubo_value
#generate the bqm
bqm = dimod.make_quadratic(HUBO, strength, dimod.BINARY)

In [31]:
def find_bond_theta_soln(solution):
    thetas = generate_thetas()
    bond_theta_soln = {}
    for key, value in solution.items():
        if solution[key] == 1:
            bond_theta_soln[key//n_angles] = thetas[key%n_angles]
    return bond_theta_soln

In [41]:
sampler = neal.SimulatedAnnealingSampler()
sample_size=10
start = time.time()
sampleset = sampler.sample(bqm, num_reads=sample_size)
end = time.time()
print("The time of execution: ", (end-start), "s")
print(sampleset)

The time of execution:  0.010003328323364258 s
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15      energy num_oc.
6  0  0  0  0  0  0  0  1  0  0  0  0  1  0  0  0 -943.519398       1
3  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0 -943.188778       1
4  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0 -943.188778       1
0  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0 -942.860134       1
9  0  0  1  0  0  0  0  0  0  0  0  0  0  0  1  0 -828.945461       1
5  0  1  0  0  0  0  0  0  0  1  0  0  0  0  0  0 -802.699143       1
2  0  0  1  0  0  0  0  0  0  1  0  0  0  0  0  0 -770.117914       1
7  0  0  0  0  1  0  0  0  0  0  0  0  0  0  1  0 -750.618036       1
8  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  1 -721.022114       1
1  0  0  0  1  0  0  0  0  1  0  0  0  0  0  0  0 -700.250176       1
['BINARY', 10 rows, 10 samples, 16 variables]


In [42]:
sa_soln = sampleset.first.sample
print(sa_soln)
bond_theta_soln = find_bond_theta_soln(sa_soln)
print(bond_theta_soln)

{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 1, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0}
{0: 7*pi/4, 1: pi}


In [44]:
sampler = EmbeddingComposite(DWaveSampler())
start = time.time()
sampleset = sampler.sample(bqm, num_reads=1000)
end = time.time()
print("The time of execution: ", (end-start), "s")
print(sampleset.slice(10))

The time of execution:  0.18275666236877441 s
   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15      energy num_oc. ...
0  1  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0 -975.906954      19 ...
1  0  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0 -975.770007      14 ...
2  0  0  0  0  0  0  0  1  0  0  0  0  1  0  0  0 -943.519398       6 ...
3  0  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0 -943.188778       9 ...
4  1  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0 -942.860134       7 ...
5  0  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0 -942.723187       9 ...
6  1  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0 -938.254395       4 ...
7  0  1  0  0  0  0  0  0  0  0  0  0  0  1  0  0 -938.117448       3 ...
8  0  0  0  0  0  0  0  1  0  0  0  1  0  0  0  0 -910.472578       6 ...
9  0  0  1  0  0  0  0  0  0  0  0  1  0  0  0  0 -910.141958      13 ...
['BINARY', 10 rows, 90 samples, 16 variables]


In [45]:
qa_soln = sampleset.first.sample
print(qa_soln)
bond_theta_soln = find_bond_theta_soln(qa_soln)
print(bond_theta_soln)

{0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0}
{0: 0, 1: pi}


In [46]:
def rotation_matrix_new_coords(first_coords, second_coords, bond_no, soln_theta):
    x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
    x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
    dx = x_ddash - x_dash
    dy = y_ddash - y_dash
    dz = z_ddash - z_dash
    l_sq = dx ** 2 + dy ** 2 + dz ** 2
    l = sp.sqrt(l_sq)
    c_theta = sp.cos(soln_theta)
    s_theta = sp.sin(soln_theta)
    index = n_angles * bond_no
    rotation_matrix = eye(4)
    rotation_matrix[0, 0] = ((dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq).evalf()
    rotation_matrix[0, 1] = ((dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq).evalf()
    rotation_matrix[0, 2] = ((dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq).evalf()
    rotation_matrix[0, 3] = (((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (
            y_dash * dz - z_dash * dy) * l * s_theta) / l_sq).evalf()
    rotation_matrix[1, 0] = ((dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq).evalf()
    rotation_matrix[1, 1] = ((dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq).evalf()
    rotation_matrix[1, 2] = ((dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq).evalf()
    rotation_matrix[1, 3] = (((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (
            z_dash * dx - x_dash * dz) * l * s_theta) / l_sq).evalf()
    rotation_matrix[2, 0] = ((dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq).evalf()
    rotation_matrix[2, 1] = ((dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq).evalf()
    rotation_matrix[2, 2] = ((dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq).evalf()
    rotation_matrix[2, 3] = (((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (
            x_dash * dy - y_dash * dx) * l * s_theta) / l_sq).evalf()
    return rotation_matrix

In [84]:
final_coords = copy.deepcopy(coords_original)

coords_rotation_dict_final = {}
n_atoms_final = mol.GetNumAtoms()
for i in range(0, 11):
    coords_rotation_dict_final[i] = []
for i in range(11, 21):
    coords_rotation_dict_final[i] = [0]
for i in range(21, 25):
    coords_rotation_dict_final[i] = [0, 1]

print(coords_rotation_dict_final)

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [0], 12: [0], 13: [0], 14: [0], 15: [0], 16: [0], 17: [0], 18: [0], 19: [0], 20: [0], 21: [0, 1], 22: [0, 1], 23: [0, 1], 24: [0, 1]}


In [85]:
torsional_bonds_final = {
    0: (9, 10),
    1: (19, 20)
}

In [86]:
def compute_new_coordinates():
    for i in coords_original.keys():
        if len(coords_rotation_dict_final[i]) > 0:
            rot_mat = eye(4, 4)
            for bond_no in coords_rotation_dict_final[i]:
                temp_rot_mat = rotation_matrix_new_coords(coords_original[torsional_bonds_final[bond_no][0]],
                                                        coords_original[torsional_bonds_final[bond_no][1]], bond_no, bond_theta_soln[bond_no])
                rot_mat = rot_mat * temp_rot_mat
            final_coords[i] = rotate_coordinates(rot_mat, coords_original[i])

In [99]:
# distance_pairs_final_indices = {
#     2: [14, 19, 20, 23],
#     6: [14, 19, 20, 23],
#     9: [14, 19, 20, 23],
#     10: [20, 23],
#     14: [20, 23],
#     19: [23]
# }

distance_pairs_final = []
# for x, y in distance_pairs_final_indices.items():
#     for i in y:
#         distance_pairs_final.append((x, i))
for i in range(n_atoms_final):
    for j in range(i+1, n_atoms_final):
        distance_pairs_final.append((i,j))
print(distance_pairs_final)

[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23), (2, 24), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (3, 17), (3, 18), (3, 19), (3, 20), (3, 21), (3, 22), (3, 23), (3, 24), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (4, 14), (4, 15), (4, 16), (4, 17), (4, 18), (4, 19), (4, 20), (4, 21), (4, 22), (4, 23), (4, 24), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5,

In [129]:
def find_volume_change():
    old_distance_sq = 0
    for pair in distance_pairs_final:
        old_distance_sq += distance_squared(coords_original[pair[0]], coords_original[pair[1]])
    # print("old volume: ", old_distance_sq)
    distance_sq = 0
    for pair in distance_pairs_final:
        distance_sq += distance_squared(final_coords[pair[0]], final_coords[pair[1]])
    # print("new volume: ", distance_sq.evalf())
    # print('change in volume: ', distance_sq.evalf() - old_distance_sq)
    return distance_sq.evalf() - old_distance_sq

In [167]:
bond_theta_soln = {0: 0, 1: sp.pi}
# bond_theta_soln = {0: sp.pi/4}
# bond_theta_soln = {0: 2*sp.pi/4}
# bond_theta_soln = {0: 3*sp.pi/4}
# bond_theta_soln = {0: 4*sp.pi/4}
# bond_theta_soln = {0: 5*sp.pi/4}
# bond_theta_soln = {0: 6*sp.pi/4}
# bond_theta_soln = {0: 7*sp.pi/4}

In [168]:
sp.pprint(coords_original)

{0: [12.305, -0.778, -0.508], 1: [11.0, -0.01, -0.373], 2: [9.839, -0.938, -0.
008], 3: [8.526, -0.154, 0.13], 4: [7.333, -1.002, 0.304], 5: [7.385, -1.885, 
1.482], 6: [7.42, -1.187, 2.849], 7: [7.463, -2.211, 3.987], 8: [7.494, -1.531
, 5.347], 9: [6.114, -0.178, 0.22], 10: [4.808, -0.978, 0.227], 11: [3.609, -0
.075, -0.057], 12: [2.435, -0.883, 0.009], 13: [1.208, -0.308, -0.214], 14: [1
.06, 1.058, -0.506], 15: [-0.199, 1.601, -0.706], 16: [-1.31, 0.782, -0.608], 
17: [-1.17, -0.587, -0.317], 18: [-2.677, -1.439, -0.213], 19: [0.095, -1.206,
 -0.116], 20: [0.35, -2.78, 0.217], 21: [-0.849, -3.648, 0.219], 22: [1.181, -
3.165, 1.47], 23: [2.134, -3.924, 0.685], 24: [1.449, -3.574, -0.544]}


In [169]:
compute_new_coordinates()
sp.pprint(final_coords)

{0: [12.305, -0.778, -0.508], 1: [11.0, -0.01, -0.373], 2: [9.839, -0.938, -0.
008], 3: [8.526, -0.154, 0.13], 4: [7.333, -1.002, 0.304], 5: [7.385, -1.885, 
1.482], 6: [7.42, -1.187, 2.849], 7: [7.463, -2.211, 3.987], 8: [7.494, -1.531
, 5.347], 9: [6.114, -0.178, 0.22], 10: [4.808, -0.978, 0.227], 11: [3.609, -0
.075, -0.057], 12: [2.435, -0.883, 0.009], 13: [1.208, -0.308, -0.214], 14: [1
.06, 1.058, -0.506], 15: [-0.199, 1.601, -0.706], 16: [-1.31, 0.782, -0.608], 
17: [-1.17, -0.587, -0.317], 18: [-2.677, -1.439, -0.213], 19: [0.095, -1.206,
 -0.116], 20: [0.35, -2.78, 0.217], 21: [1.75296098198908, -3.17095915941494, 
0.481349047068091], 22: [-0.243596738511866, -3.86038326895029, -0.72597927029
1966], 23: [-0.970508315777176, -4.49692514104598, 0.354265611161571], 24: [-0
.503630913661392, -3.50055271332145, 1.29842315980689]}


In [170]:
find_volume_change()

733.573211896271

In [146]:
# brute force solution
start = time.time()
thetas = generate_thetas()
best_volume_change = 0
best_soln = {}
for i in range(8):
    for j in range(8):
        bond_theta_soln = {0: thetas[i], 1: thetas[j]}
        compute_new_coordinates()
        if find_volume_change() > best_volume_change:
            best_volume_change = find_volume_change()
            best_soln = bond_theta_soln
end = time.time()
print("The time of execution: ", (end-start), "s")    
print('best solution: ', best_soln)




The time of execution:  17.2250919342041 s
best solution:  {0: 0, 1: pi}


In [171]:
from rdkit.Geometry import Point3D
conf = mol.GetConformer()
for i in range(n_atoms_final):
    x, y, z = final_coords[i]
    conf.SetAtomPosition(i, Point3D(float(x), float(y), float(z)))

In [172]:
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [173]:
mol_file = './Molecules/25A12Rot_2.mol2'
mol = Chem.MolFromMol2File(mol_file)
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol